In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.optimizers import Adam
from kerastuner.tuners import RandomSearch
from keras.initializers import Constant
from sklearn.preprocessing import LabelEncoder



In [62]:
import numpy as np
from keras.callbacks import LearningRateScheduler


In [63]:
#preprocessinmg the data sets

In [64]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv", header=None)

In [65]:
train_data.head(5)

,Tweets,label
0,Asked #ChatGPT about what it thinks are the pr...,Positive
1,#ChatGPT tornado has already traveled around t...,Neutral
2,This is a great explanation of why #EVs are mo...,Positive
3,‘if you need to write a box-ticking social med...,Positive
4,Just saw an AI tool making my coffee for me. \...,Positive


In [66]:
train_data.isna().sum()

Tweets    171
label     171
dtype: int64

In [67]:
train_data.dropna(inplace=True)  # Remove any rows with missing values

In [68]:
train_data.head(5)

,Tweets,label
0,Asked #ChatGPT about what it thinks are the pr...,Positive
1,#ChatGPT tornado has already traveled around t...,Neutral
2,This is a great explanation of why #EVs are mo...,Positive
3,‘if you need to write a box-ticking social med...,Positive
4,Just saw an AI tool making my coffee for me. \...,Positive


In [69]:
train_data.isna().sum()

Tweets    0
label     0
dtype: int64

In [70]:
label_encoder = LabelEncoder()
train_data['label'] = label_encoder.fit_transform(train_data['label'])

In [71]:
X_train = train_data['Tweets']
y_train = train_data['label']

In [72]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_train_pad = pad_sequences(X_train_seq)

In [73]:
def lr_scheduler(epoch, lr):
    if epoch % 5 == 0 and epoch != 0:
        lr = lr * 0.1
    return lr

In [74]:
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 100, input_length=X_train_pad.shape[1]))
model.add(LSTM(units=128, dropout=0.5, recurrent_dropout=0.5))  # Increased dropout rate
model.add(Dense(3, activation='softmax'))

In [75]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [76]:
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train_pad, y_train, test_size=0.2, random_state=42)

In [77]:
history = model.fit(X_train_final, to_categorical(y_train_final), epochs=10, batch_size=32, validation_data=(X_val, to_categorical(y_val)), callbacks=[LearningRateScheduler(lr_scheduler)])


Epoch 1/10
15/15 [==============================] - 3s 75ms/step - loss: 1.0030 - accuracy: 0.6096 - val_loss: 0.9430 - val_accuracy: 0.5917 - lr: 0.0010
Epoch 2/10
15/15 [==============================] - 1s 54ms/step - loss: 0.9079 - accuracy: 0.6409 - val_loss: 0.9437 - val_accuracy: 0.5917 - lr: 0.0010
Epoch 3/10
15/15 [==============================] - 1s 52ms/step - loss: 0.8635 - accuracy: 0.6409 - val_loss: 0.9398 - val_accuracy: 0.5917 - lr: 0.0010
Epoch 4/10
15/15 [==============================] - 1s 51ms/step - loss: 0.7897 - accuracy: 0.6409 - val_loss: 0.9174 - val_accuracy: 0.5917 - lr: 0.0010
Epoch 5/10
15/15 [==============================] - 1s 50ms/step - loss: 0.5834 - accuracy: 0.7244 - val_loss: 0.8894 - val_accuracy: 0.6083 - lr: 0.0010
Epoch 6/10
15/15 [==============================] - 1s 51ms/step - loss: 0.4046 - accuracy: 0.8810 - val_loss: 0.8475 - val_accuracy: 0.6000 - lr: 1.0000e-04
Epoch 7/10
15/15 [==============================] - 1s 53ms/step - loss:

In [78]:
X_test_seq = tokenizer.texts_to_sequences(test_data[0])
X_test_pad = pad_sequences(X_test_seq, maxlen=X_train_pad.shape[1])

In [60]:
test_labels_encoded = label_encoder.transform(test_data[1])

ValueError: y contains previously unseen labels: 'Positive'

In [59]:

loss, accuracy = model.evaluate(X_test_pad, to_categorical(label_encoder.transform(test_data[1])))
print("Test Accuracy:", accuracy)

ValueError: y contains previously unseen labels: 'Positive'

In [58]:
# Assuming label_encoder is the same instance used during training
test_labels_encoded = label_encoder.transform(test_data[1])
# Evaluate model on test data
loss, accuracy = model.evaluate(X_test_pad, to_categorical(test_labels_encoded))
print("Test Accuracy:", accuracy)


ValueError: y contains previously unseen labels: 'Positive'